# Product Recommendation for Customer

> Sistem Rekomendasi yang akan diberikan ke customer pada case kali ini adalah sistem rekomendasi berdasarkan **demographic filtering** dan **colaborative filtering** (item based), dengan detail rekomendasi sebagai berikut:
- top produk berdasarkan kategori pilihan customer
- kategori lain yang mungkin akan disukai oleh customer berdasarkan histori user sebelumnya

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from pandas import Series
import itertools
import warnings                 
warnings.filterwarnings("ignore")

In [2]:
df_cust = pd.read_csv('olist_customers_dataset.csv')
df_geo = pd.read_csv('olist_geolocation_dataset.csv')
df_order = pd.read_csv('olist_order_items_dataset.csv')
df_payment = pd.read_csv('olist_order_payments_dataset.csv')
df_review = pd.read_csv('olist_order_reviews_dataset.csv') #belum clean
df_delivery = pd.read_csv('olist_orders_dataset.csv')
df_product = pd.read_csv('olist_products_dataset.csv') #belum clean
df_seller = pd.read_csv('olist_sellers_dataset.csv')
df_catprod = pd.read_csv('product_category_name_translation.csv')

# Pre Processing Data

Data preprocessing dilakukan untuk mendapatkan data yang memiliki informasi sebagai berikut:
- customer
- product id
- product category
- review score

In [3]:
df_prod_eng = df_product.merge(df_catprod,on='product_category_name',how='outer')[['product_id',
                                                                     'product_category_name_english','product_category_name','product_name_lenght',
                                                                     'product_description_lenght','product_photos_qty',
                                                                     'product_weight_g','product_length_cm',
                                                                     'product_height_cm','product_width_cm']]

In [4]:
def applyCat(x):
    if pd.notnull(x['product_category_name_english']):
        return x['product_category_name_english']
    else:
        return x['product_category_name']

def applyCatName(x):
    if (x['product_category_name_english']=='portateis_cozinha_e_preparadores_de_alimentos'):
        return 'portable_cooking_and_food_prepareers'
    else:
        return x['product_category_name_english']
    
df_prod_eng['product_category_name_english'] = df_prod_eng.apply(applyCat,axis=1)
df_prod_eng['product_category_name_english'] = df_prod_eng.apply(applyCatName,axis=1)
df_prod_eng.drop(columns='product_category_name',inplace=True)
df_prod_eng.dropna(inplace=True)

In [5]:
df_recom = df_prod_eng.merge(df_order, left_on='product_id', 
                  right_on='product_id')[['product_id','product_category_name_english',
                                          'product_name_lenght','product_description_lenght','product_photos_qty',
                                          'product_weight_g','product_length_cm','product_height_cm',
                                          'product_width_cm','order_item_id','order_id']]

df_recom = df_recom.merge(df_review, left_on='order_id', 
                                right_on='order_id')[['product_id','product_category_name_english',
                                                      'product_name_lenght','product_description_lenght',
                                                      'product_photos_qty','product_weight_g','product_length_cm',
                                                      'product_height_cm','product_width_cm','order_item_id',
                                                      'order_id','review_score']]

df_recom = df_recom.drop(columns=['product_name_lenght','product_description_lenght','product_photos_qty',
                       'product_weight_g','product_length_cm','product_height_cm','product_width_cm'])

df_recom = df_recom.merge(df_delivery, left_on='order_id', 
               right_on='order_id').drop(columns=['order_purchase_timestamp','order_approved_at',
                                                  'order_delivered_carrier_date','order_delivered_customer_date',
                                                  'order_estimated_delivery_date','order_status'])

df_prodsel = df_order[['order_id','seller_id']]
df_recom = df_recom.merge(df_prodsel, on='order_id')
df_recom = df_recom[['customer_id','product_id','product_category_name_english','seller_id','review_score']]
df_recom.head()

,customer_id,product_id,product_category_name_english,seller_id,review_score
0,f8a3e963a310aa58b60a5b1fed5bceb5,1e9e8ef04dbcff4541ed26657ea517e5,perfumery,5670f4db5b62c43d542e1b2d56b0cf7c,5
1,12a2c614dfc8926e793516b1638c7471,6a2fb4dd53d2cdb88e0432f1284a004c,perfumery,7040e82f899a04d1b434b795a43b4617,5
2,c15eed3881abc9e06763c989679c5094,6a2fb4dd53d2cdb88e0432f1284a004c,perfumery,7040e82f899a04d1b434b795a43b4617,2
3,6d699e3ab0fd90fffb3d1379d937de77,0d009643171aee696f4733340bc2fdd0,perfumery,fe2032dab1a61af8794248c8196565c9,5
4,45d942b1af5529f8990f27848a40e645,0d009643171aee696f4733340bc2fdd0,perfumery,7178f9f4dd81dcef02f62acdf8151e01,3


In [6]:
df_recom.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 156140 entries, 0 to 156139
Data columns (total 5 columns):
customer_id                      156140 non-null object
product_id                       156140 non-null object
product_category_name_english    156140 non-null object
seller_id                        156140 non-null object
review_score                     156140 non-null int64
dtypes: int64(1), object(4)
memory usage: 7.1+ MB


# Content Based

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
df_cat_score = df_recom[['product_id','product_category_name_english', 'review_score']]
df_score_mean = pd.DataFrame(df_cat_score.groupby('product_id')['review_score'].mean())
df_cat_score = df_score_mean.merge(df_recom[['product_id','product_category_name_english']],on='product_id',how='outer').drop_duplicates(subset='product_id')
df_cat_score = df_cat_score.sort_values(by='review_score',ascending=False)
df_cat_score=df_cat_score.reset_index()
df_cat_score.drop(columns=['index'],inplace=True)

# Demographic Filtering
> Pada Demographic Filtering akan ditampilkan top product berdasarkan rating dan category yang dipilih oleh customer

In [9]:
df_demo = df_cat_score[['product_category_name_english','product_id','review_score']]

In [10]:
def demo(x):
    table = []
    for item in x:
        new_table = df_demo[df_demo['product_category_name_english']==item].sort_values(by='review_score',ascending=False).head(5)
        table.append(new_table)
    return table

In [24]:
for item in (demo(['perfumery','health_beauty','housewares'])):
    display(item)

,product_category_name_english,product_id,review_score
0,perfumery,00066f42aeeb9f3007548bb9d3f33c38,5.0
9328,perfumery,e57b2d6510f090860888171c76b42b21,5.0
9475,perfumery,2829a5eb8a0ac813672891b4c91f7c65,5.0
9464,perfumery,2800a94a39d630f1713089675b8350a9,5.0
9438,perfumery,287398c3a5da555d519e464b674c1955,5.0


,product_category_name_english,product_id,review_score
3,health_beauty,c5588523a52b41fd7e7340375e67b6cf,5.0
8915,health_beauty,1a432f73243abc5629393e03b5540da9,5.0
8961,health_beauty,f08d78a6e7eecc6f3f51fad33ce7be1b,5.0
8959,health_beauty,f08d9c15742df022236141f83e0a8a22,5.0
8954,health_beauty,188372a703d9a84ec0b6a32a0044c7f2,5.0


,product_category_name_english,product_id,review_score
19,housewares,5ef85fbb12f3f0fef4e3f96985c092a0,5.0
9109,housewares,e7ce29342574abe2fbf171df97fdb092,5.0
8562,housewares,1dd3a0b856546758b41e8ef9935f1831,5.0
8567,housewares,1de0af19bfefbea493eb82ef59bb2433,5.0
8570,housewares,ed47a454f5fbb920098724ccf04f86de,5.0


# Collaborative Rating (Item Based)

> Pada collaborative rating (item based) akan diberikan rekomendasi kategori lain yang mungkin akan disukai oleh customer berdasarkan histori user sebelumnya

In [12]:
df_recom_ib = df_recom.reset_index().drop(columns='index')
df_recom_ib = df_recom_ib.drop(columns='seller_id')

In [13]:
def apply_cust(x):
    if pd.notnull(x['customer_id']):
        return (x['customer_id'][-4:])

In [14]:
df_recom_ib['customer_id'] = df_recom_ib.apply(apply_cust,axis=1)
df_ib_pivot = df_recom_ib.pivot_table(index='product_category_name_english',columns='customer_id',values='review_score')
df_ib_pivot.fillna(0,inplace=True)

In [15]:
similarity_product = cosine_similarity(df_ib_pivot)
df_item_similarity = pd.DataFrame(similarity_product,
                                  index=df_ib_pivot.index,columns=df_ib_pivot.index)

In [16]:
def similarity(x):
    top_list = []
    for item in x:
        top = list(df_item_similarity[item].sort_values(ascending=False)[1:4].index)
        top_list.append(top)
    table = []
    for item in top_list:
        for elemen in item:
            new_table = df_demo[df_demo['product_category_name_english']==elemen].sort_values(by='review_score',ascending=False).head(3)
            table.append(new_table)
    return table

In [17]:
df_item_similarity['agro_industry_and_commerce'].sort_values(ascending=False).head()

product_category_name_english
agro_industry_and_commerce    1.000000
housewares                    0.019415
bed_bath_table                0.019174
construction_tools_safety     0.017735
health_beauty                 0.017031
Name: agro_industry_and_commerce, dtype: float64

In [18]:
for item in (similarity(['agro_industry_and_commerce','health_beauty','housewares'])):
    display(item)

,product_category_name_english,product_id,review_score
19,housewares,5ef85fbb12f3f0fef4e3f96985c092a0,5.0
9109,housewares,e7ce29342574abe2fbf171df97fdb092,5.0
8562,housewares,1dd3a0b856546758b41e8ef9935f1831,5.0


,product_category_name_english,product_id,review_score
1,bed_bath_table,5ef15970297e8282ea1f0e3ddc528503,5.0
9131,bed_bath_table,e716e067c266c9785c18204cb3592422,5.0
8958,bed_bath_table,f092d3abbc2f3281c56b4497e5c150f0,5.0


,product_category_name_english,product_id,review_score
396,construction_tools_safety,6054d161235b97a4aaccea2a086d30b8,5.0
11613,construction_tools_safety,0b60c390c7176d836fa228baa164af0d,5.0
8842,construction_tools_safety,19dcf909566fad3eb56a49fa80193852,5.0


,product_category_name_english,product_id,review_score
1,bed_bath_table,5ef15970297e8282ea1f0e3ddc528503,5.0
9131,bed_bath_table,e716e067c266c9785c18204cb3592422,5.0
8958,bed_bath_table,f092d3abbc2f3281c56b4497e5c150f0,5.0


,product_category_name_english,product_id,review_score
49,sports_leisure,5eaa343860dc445b3fd43d1b682809fd,5.0
9057,sports_leisure,1909929bf0f282504162cf7e8290c4bd,5.0
9098,sports_leisure,26679d24a4e81ddc13cce48ff1a9d001,5.0


,product_category_name_english,product_id,review_score
12,computers_accessories,5ee52854d55c499ec82dc779c98fa02c,5.0
9715,computers_accessories,1fe7c14d5b10473294330ae485559ea1,5.0
9321,computers_accessories,29f5540c79dd4bb8f156df4e1166b9f4,5.0


,product_category_name_english,product_id,review_score
49,sports_leisure,5eaa343860dc445b3fd43d1b682809fd,5.0
9057,sports_leisure,1909929bf0f282504162cf7e8290c4bd,5.0
9098,sports_leisure,26679d24a4e81ddc13cce48ff1a9d001,5.0


,product_category_name_english,product_id,review_score
1,bed_bath_table,5ef15970297e8282ea1f0e3ddc528503,5.0
9131,bed_bath_table,e716e067c266c9785c18204cb3592422,5.0
8958,bed_bath_table,f092d3abbc2f3281c56b4497e5c150f0,5.0


,product_category_name_english,product_id,review_score
46,furniture_decor,5ea241885816f6957dfaa9a8592c6b37,5.0
9068,furniture_decor,26ce0f39be4f4b86ab1b75e9d5015e9d,5.0
9100,furniture_decor,e7f539de944d2d7d5c206cd730f606a4,5.0


# Export data with Pickle (for flask)

In [19]:
import pickle

In [20]:
filename = 'prod_cat_colomn.sav'
pickle.dump(df_recom_ib['product_category_name_english'].unique(), open(filename , 'wb'))

In [21]:
filename = 'df_demo.sav'
pickle.dump(df_demo, open(filename , 'wb'))

In [22]:
filename = 'similarity.sav'
pickle.dump(df_item_similarity, open(filename , 'wb'))